In [1]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE

/Users/CPang/Cecil/PhD/news-outlets/venv/lib/python3.10/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/CPang/Cecil/PhD/news-outlets/venv/lib/python3.10/site-packages/libpyg.so, 0x0006): Library not loaded: /usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python
  Referenced from: <0693525D-A908-3909-8C99-435019B5A8B8> /Users/CPang/Cecil/PhD/news-outlets/venv/lib/python3.10/site-packages/libpyg.so
  Reason: tried: '/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file, not in dyld cache)
  warnings.warn(f"An issue occurred while 

In [2]:
import torch_geometric
print(torch_geometric.typing.WITH_PYG_LIB)

False


In [3]:
import pyg_lib

OSError: dlopen(/Users/CPang/Cecil/PhD/news-outlets/venv/lib/python3.10/site-packages/libpyg.so, 0x0006): Library not loaded: /usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python
  Referenced from: <0693525D-A908-3909-8C99-435019B5A8B8> /Users/CPang/Cecil/PhD/news-outlets/venv/lib/python3.10/site-packages/libpyg.so
  Reason: tried: '/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/usr/local/opt/python@3.12/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file, not in dyld cache)

In [6]:
dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath('__file__')), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [7]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [8]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [9]:
train_loader = LinkNeighborLoader(
    data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device, 'x', 'edge_index')

model = GraphSAGE(
    data.num_node_features,
    hidden_channels=64,
    num_layers=2,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
model.train()

total_loss = 0
for batch in train_loader:
    batch = batch.to(device)
    optimizer.zero_grad()
    h = model(batch.x, batch.edge_index)
    h_src = h[batch.edge_label_index[0]]
    h_dst = h[batch.edge_label_index[1]]
    pred = (h_src * h_dst).sum(dim=-1)
    loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
    loss.backward()
    optimizer.step()
    total_loss += float(loss) * pred.size(0)
    break

ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

In [ ]:
def train():
    model.train()

    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        h_src = h[batch.edge_label_index[0]]
        h_dst = h[batch.edge_label_index[1]]
        pred = (h_src * h_dst).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)

    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index).cpu()

    clf = LogisticRegression()
    clf.fit(out[data.train_mask], data.y[data.train_mask])

    val_acc = clf.score(out[data.val_mask], data.y[data.val_mask])
    test_acc = clf.score(out[data.test_mask], data.y[data.test_mask])

    return val_acc, test_acc


times = []
for epoch in range(1, 51):
    start = time.time()
    loss = train()
    val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")